# Diccionarios

- Es una estructura de datos que funcionan como un conjunto de pares llave:valor

In [1]:
dicc_ingles = {'azul':'blue', 'casa':'house', 'ratón':'mouse'}
print(dicc_ingles['azul'])

blue


In [3]:
dicc_ingles[3] = [4.32, "m", True, 0]

In [4]:
print(dicc_ingles[3])

[4.32, 'm', True, 0]


- Las llaves y los valores pueden ser otras estructuras de datos anidadas
- Si se intenta acceder una llave que no existe, se genera un error
- Para evitar esto, podemos utilizar el operador "in", que retorna True si existe la llave en el diccionario y False si no

In [5]:
print('computadora' in dicc_ingles)
print('azul' in dicc_ingles)

False
True


- Se puede iterar sobre las llaves de un diccionario:

In [6]:
for patito in dicc_ingles:
    print(patito)

azul
casa
ratón
3


- Y sobre los valores:

In [7]:
for llave in dicc_ingles:
    print (dicc_ingles[llave])

blue
house
mouse
[4.32, 'm', True, 0]


# NetCDF
- Sus siglas significan Network Common Data Form
- La idea es tener un formato de datos independiente de la máquina donde esté
- Orientado a datos científicos
- Usualmente utilizado en los campos de la meteorología, climatología, oceanografía y aplicaciones de SIG
- Es un estándar abierto

La clase $\textbf{Dataset}$ de la biblioteca NetCDF es una colección de dimensiones, grupos, variables y atributos. De forma conjunta describen los datos y las relaciones entre los distintos tipos de datos 

- Para crear un archivo NetCDF desde Python, se llama al constructor de la clase Dataset
- Este método se llama también para abrir un archivo NetCDF (en adelante $\textbf{nc}$) existente
- El constructor de la clase $\textbf{Dataset}$ recibe varios parámetros:
    - El nombre del archivo que se desea abrir o crear
    - El modo en el que se desea abrir el archivo (son los mismos modos que se utilizan cuando se abren archivos normales en Python)
    - Formato (opcional)
    

In [10]:
from netCDF4 import Dataset
set_datos = Dataset("test.nc", "w", format="NETCDF4")
set_datos.close()

- Los datos dentro de un archivo $\textbf{nc}$ se pueden ordenar jerárquicamente en grupos análogo a como se ordenan directorios en un sistema de archivos
- Cada $\textbf{Dataset}$ tiene un grupo especial llamado "grupo raíz"
- Para crear grupos adicionales se utiliza el método createGroup. Este método recibe como parámetro un texto que es el identificador del grupo
- Los grupos creados se pueden visualizar como un $\textbf{diccionario}$

In [11]:
set_datos = Dataset("test.nc", "a")
grupo_pron = set_datos.createGroup("pronosticos")
grupo_analis = set_datos.createGroup("analisis")
print (set_datos.groups)

OrderedDict([('pronosticos', <class 'netCDF4._netCDF4.Group'>
group /pronosticos:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
), ('analisis', <class 'netCDF4._netCDF4.Group'>
group /analisis:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
)])


- Los grupos se pueden anidar unos dentro de otros

In [12]:
grupo1 = set_datos.createGroup("/pronosticos/modelo1")
grupo2 = set_datos.createGroup("/pronosticos/modelo2")

- Los archivos NetCDF definen los tamaños de todas sus variables en términos de dimensiones
- Antes de crear cualquier variable, se deben de crear dimensiones
- Se crean con el método $\textbf{createDimension}$, que recibe el texto del nombre de la dimensión y un valor entero para indicar el tamaño de la dimensión, si se desea crear una dimensión sin límite, se le puede poner None o 0 a este segundo parámetro

In [14]:
nivel = set_datos.createDimension("nivel", None)
tiempo = set_datos.createDimension("tiempo", None)
lat = set_datos.createDimension("lat", 73)
lon = set_datos.createDimension("lon", 144)

- Todas las instancias de dimensiones creadas se almacenan también en un diccionario

In [15]:
print(set_datos.dimensions)

OrderedDict([('nivel', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'nivel', size = 0
), ('tiempo', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'tiempo', size = 0
), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73
), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 144
)])


# Variables
- Las variables de $\textbf{nc}$ se comportan muy parecido a los arrays de NumPy, con la diferencia de que no necesariamente tienen una cantidad fija de datos (si se crea la dimensión sin límite)
- Para crear variables, se utiliza el método $\textbf{createVariable}$ de la clase $\textbf{Dataset}$ o de una instancia de $\textbf{grupo}$. Este método tiene 2 parámetros obligatorios, el nombre de la variable, y el tipo de dato que es
- Se le puede también indicar una tupla con las dimensiones de la variable, pero no es necesario. Si este parámetro no se especifica, la variable se toma como escalar

In [17]:
import numpy as np
niveles = set_datos.createVariable('nivel', np.int32, ('nivel',))
tiempos = set_datos.createVariable('tiempo', np.float64, ('tiempo',))
latitudes = set_datos.createVariable('latitud', np.float32,('lat',))
longitudes = set_datos.createVariable('longitud', np.float32,('lon',))

# Variable con las 4 dimensiones:
temp = set_datos.createVariable('temp', np.float32,('nivel','tiempo','lat','lon')) 

## Acceso a las variables
- Las variables en un $\textbf{Dataset}$ se almacenan también en un diccionario de Python

In [2]:
for nomb_var in set_datos.variables.keys():
    variable = set_datos.variables[nomb_var]
    print(nomb_var, variable.dtype, variable.dimensions, variable.shape)

lat float32 ('lat',) (73,)
lon float32 ('lon',) (144,)
time float64 ('time',) (366,)
air int16 ('time', 'lat', 'lon') (366, 73, 144)


# Ejercicio 
### Cargue el archivo llamado "air.nc" (o si tiene otro archivo NetCDF, puede utilizarlo), imprima en consola cuáles dimensiones y cuáles variables tiene el archivo

# Solución

In [19]:
aire = Dataset("air.nc", "r")
print("---------- dimensiones ----------- ")
for dimensiones in aire.dimensions.keys():
    print(dimensiones)
print("---------- variables ----------- ")
for variables in aire.variables.keys():
    print(variables)
    
aire.close()

---------- dimensiones ----------- 
lat
lon
time
---------- variables ----------- 
lat
lon
time
air


# Ingreso de datos

- Una vez que se cuenta con una instancia de una variable, se le puede ingresar datos
- Se le trata como a un numpy array. Es decir que se le puede aplicar todas las mismas funciones que a un array

In [20]:
lats = np.arange(8,12,0.1)
lons = np.arange(-86,-82,0.1) 
set_datos.variables["latitud"] = lats
set_datos.variables["longitud"] = lons
print (set_datos.variables["longitud"])

[-86.  -85.9 -85.8 -85.7 -85.6 -85.5 -85.4 -85.3 -85.2 -85.1 -85.  -84.9
 -84.8 -84.7 -84.6 -84.5 -84.4 -84.3 -84.2 -84.1 -84.  -83.9 -83.8 -83.7
 -83.6 -83.5 -83.4 -83.3 -83.2 -83.1 -83.  -82.9 -82.8 -82.7 -82.6 -82.5
 -82.4 -82.3 -82.2 -82.1]


# Una vez que se termina de utilizar el Dataset, debe de cerrarse para guardar todos los cambios en el archivo

In [21]:
set_datos.close()

# Utilización de Basemap para visualizar datos

In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

mapa = Basemap(projection='stere',lon_0=lons[0],lat_0=lats[0],lat_ts=lats[0], llcrnrlat=lats[0],urcrnrlat=lats[-1],\
            llcrnrlon=lons[0],urcrnrlon=lons[-1], rsphere=6370997.,resolution='i',area_thresh=10000)

mapa.drawmapboundary(fill_color='aqua')
mapa.fillcontinents(color='coral',lake_color='aqua')
mapa.drawcoastlines()
mapa.drawcountries()

x, y = mapa(lats, lons)

mapa.plot(x, y, marker='D',color='m')

plt.show()

# Utilizando un mapa para proyectar los datos obtenidos desde el archivo nc

In [ ]:
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid

nc = Dataset("air.nc", "r")
lats = nc.variables['lat'][:]  
lons = nc.variables['lon'][:]
time = nc.variables['time'][:]
air = nc.variables['air'][:]

mapa = Basemap(projection='moll', llcrnrlat=-90, urcrnrlat=90,\
            llcrnrlon=0, urcrnrlon=360, resolution='c', lon_0=0)

mapa.drawcoastlines()
mapa.drawmapboundary()

# Hacer contínuo al mapa
air_cyclic, lons_cyclic = addcyclic(air[237, :, :], lons)
# Hace que las longitudes vayan de -180 a 180 en vez de de 0 a 360.
air_cyclic, lons_cyclic = shiftgrid(180., air_cyclic, lons_cyclic, start=False)

# Crear los arrays 2D para el mapa
lon2d, lat2d = np.meshgrid(lons_cyclic, lats)

# Transforma lat/lon en coordenadas para la proyección
x, y = mapa(lon2d, lat2d)

# Plotea las temperaturas del aire con 11 intervalos de contornos
cs = mapa.contourf(x, y, air_cyclic, 11, cmap=plt.cm.Spectral_r)
cbar = plt.colorbar(cs, orientation='horizontal', shrink=0.5)

# Ejercicio
## Tomando su propio set de datos (puede descargar uno de internet), cree una visualización de ellos